# Start elastic search server first

In [2]:
from elasticsearch import Elasticsearch 
# Connect to the elastic cluster
es=Elasticsearch([{'host':'localhost','port':9200}])


# Create local index

In [3]:
es.indices.create(index='covid-document-index', ignore=400)
es.indices.create(index='covid-section-index', ignore=400)
es.indices.create(index='wiki-document-index', ignore=400)
es.indices.create(index='wiki-section-index', ignore=400)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'wiki-section-index'}

## Load covid document and sections

In [19]:
import pandas as pd
from tqdm.auto import tqdm

In [8]:
covid_questions = pd.read_csv('../data/website_questions_21032020_13h00.csv')['question'].tolist()

In [ ]:
import spacy
from spacy_langdetect import LanguageDetector
nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

In [15]:
text = 'Est ce que la toux est sèche ou grasse?'
doc = nlp(text)
# document level language detection. Think of it like average language of the document!
print(doc._.language)

{'language': 'fr', 'score': 0.9999963671987652}


In [21]:
english_covid_questions = []
for i in tqdm(covid_questions):
    try:
        if nlp(i)._.language['language'] == 'en':
            english_covid_questions.append(i)
    except:
        pass

In [23]:
from os import listdir
from os.path import isfile, join
mypath = '../scrape'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../scrape'

In [13]:
for i in tqdm(onlyfiles):
    with open(mypath + i, 'r', encoding='utf-8') as f:
        t = f.read()
        rec = {
            'file_name': i,
            'content': t
        }
        es.index('leaders-index', rec, id=int(i.replace('.txt', '')))

## Load wiki document and sections

# Search

In [4]:
from copy import deepcopy

In [57]:
all_prec_k = []
all_recall_k = []
all_hits = []
for i in tqdm(claims):
    try:
        res= es.search({'query':{
                    'match':{
                        "content": i['claim']
                    }
                }
            },index='leaders-index')
        real_ids = i['related_articles']
        ret_ids = []
        for hit in res['hits']['hits']:
            ret_ids.append(int(hit['_id']))
        all_hits.append(deepcopy(ret_ids))
    except:
        all_hits.append([-1])

In [48]:
import numpy as np

In [61]:
for K in [3, 5, 10]:
    print(K)
    all_prec_k = []
    all_recall_k = []
    for i in range(len(claims)):
        reals = claims[i]['related_articles']
        infs = all_hits[i][:K]
        rights = len(set(reals).intersection(set(infs)))
        prec_k = rights / K
        recall_k = rights / len(reals)
        all_prec_k.append(prec_k)
        all_recall_k.append(recall_k)
    print(np.mean(all_prec_k), np.mean(all_recall_k))
    print(np.std(all_prec_k), np.std(all_recall_k))
    print('**********')

3
0.3138969248901746 0.2401189496976905
0.31335288736487155 0.2664427925580082
**********
5
0.2416200578592093 0.29477200986209645
0.23806451623227468 0.2951987183216096
**********
10
0.15359691417550628 0.3601965850142953
0.14796297169687642 0.31895085993126154
**********
